# 🎥 Ultra HD Video Upscaler - 8x + 60 FPS

**Created by Md. Mahir Labib

Copyright © 2026 Md. Mahir Labib. All rights reserved.

Real-ESRGAN 8x Upscaling + RIFE AI Frame Interpolation to 60 FPS**

This notebook will:
1. **8x Upscale** your video using Real-ESRGAN (two-pass 4x upscaling)
2. **60 FPS Interpolation** using RIFE AI motion estimation
3. Output a stunning ultra HD `video-upscaled.mp4`

---

## Step 1: Check GPU & Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Install Real-ESRGAN and dependencies
!pip install -q realesrgan basicsr gfpgan facexlib
!pip install -q opencv-python-headless

# Clone and install RIFE for frame interpolation
!git clone https://github.com/hzwer/Practical-RIFE.git 2>/dev/null || echo "RIFE already cloned"

print("\n✅ All dependencies installed!")

In [ ]:
# Download Real-ESRGAN models
import os
os.makedirs('models', exist_ok=True)

# Download RealESRGAN-x4plus model (for first 4x pass)
if not os.path.exists('models/RealESRGAN_x4plus.pth'):
    print("📥 Downloading Real-ESRGAN x4 model...")
    !wget -q -P models/ https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth

# Download RealESRGAN-x2plus model (for second 2x pass to achieve 8x total)
if not os.path.exists('models/RealESRGAN_x2plus.pth'):
    print("📥 Downloading Real-ESRGAN x2 model...")
    !wget -q -P models/ https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth

# Download RIFE model
os.makedirs('Practical-RIFE/train_log', exist_ok=True)
if not os.path.exists('Practical-RIFE/train_log/flownet.pkl'):
    print("📥 Downloading RIFE model...")
    !wget -q https://github.com/hzwer/Practical-RIFE/releases/download/v4.6/flownet.pkl -O Practical-RIFE/train_log/flownet.pkl

print("✅ All models downloaded!")

## Step 2: Upload Your Video

Run the cell below to upload your video file.

In [ ]:
from google.colab import files
import shutil

print("📁 Please upload your video file...")
uploaded = files.upload()

# Get the uploaded filename
INPUT_VIDEO = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {INPUT_VIDEO}")

# Show video info
print("\n📊 Input Video Info:")
!ffprobe -v error -select_streams v:0 -show_entries stream=width,height,r_frame_rate,duration -of default=noprint_wrappers=1 "{INPUT_VIDEO}"

## Step 3: Configuration

Adjust these settings as needed:

In [ ]:
#@title ⚙️ Configuration

# Output settings
OUTPUT_VIDEO = "video-upscaled.mp4"  #@param {type:"string"}
TARGET_FPS = 60  #@param {type:"integer"}
UPSCALE_FACTOR = 8  #@param [4, 8] {type:"raw"}

# Quality settings
CRF_QUALITY = 18  #@param {type:"slider", min:10, max:28, step:1}
# Lower CRF = better quality but larger file size (18 is high quality)

# Memory optimization (reduce if OOM errors)
TILE_SIZE = 384  #@param {type:"slider", min:128, max:512, step:64}

print(f"📋 Configuration:")
print(f"   Input: {INPUT_VIDEO}")
print(f"   Output: {OUTPUT_VIDEO}")
print(f"   Upscale: {UPSCALE_FACTOR}x (Real-ESRGAN)")
print(f"   Target FPS: {TARGET_FPS} (RIFE interpolation)")
print(f"   CRF Quality: {CRF_QUALITY}")
print(f"   Tile Size: {TILE_SIZE}")

if UPSCALE_FACTOR == 8:
    print("\n🔥 8x upscale = Two-pass processing (4x → 2x)")
    print("   Example: 480p → 1920p (4x) → 3840p (8x total)")

## Step 4: Extract Frames from Video

In [ ]:
import subprocess
import glob
import os

# Create directories
for d in ['frames_lr', 'frames_4x', 'frames_8x', 'frames_interpolated']:
    os.makedirs(d, exist_ok=True)

# Get original FPS
result = subprocess.run(
    ['ffprobe', '-v', '0', '-of', 'csv=p=0', '-select_streams', 'v:0',
     '-show_entries', 'stream=r_frame_rate', INPUT_VIDEO],
    capture_output=True, text=True
)
fps_str = result.stdout.strip()
if '/' in fps_str:
    num, den = map(int, fps_str.split('/'))
    ORIGINAL_FPS = num / den
else:
    ORIGINAL_FPS = float(fps_str)

print(f"📽️ Original FPS: {ORIGINAL_FPS:.2f}")

# Extract frames
print("\n📽️ Extracting frames...")
!ffmpeg -y -i "{INPUT_VIDEO}" -qscale:v 2 frames_lr/frame_%07d.png -hide_banner -loglevel error

frame_count = len(glob.glob('frames_lr/*.png'))
print(f"✅ Extracted {frame_count} frames")

# Get original resolution
import cv2
sample = cv2.imread(glob.glob('frames_lr/*.png')[0])
orig_h, orig_w = sample.shape[:2]
print(f"📐 Original resolution: {orig_w}x{orig_h}")
print(f"🎯 Target resolution: {orig_w * UPSCALE_FACTOR}x{orig_h * UPSCALE_FACTOR}")

## Step 5: First Pass - 4x Upscale with Real-ESRGAN

In [ ]:
import cv2
import numpy as np
from tqdm.notebook import tqdm
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
import torch
import gc

# Initialize Real-ESRGAN x4
print("🔧 Initializing Real-ESRGAN x4...")

model_x4 = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)

upsampler_x4 = RealESRGANer(
    scale=4,
    model_path='models/RealESRGAN_x4plus.pth',
    model=model_x4,
    tile=TILE_SIZE,
    tile_pad=10,
    pre_pad=0,
    half=True,
    device='cuda'
)

print(f"✅ Real-ESRGAN x4 ready! Using GPU: {torch.cuda.get_device_name(0)}")

# Get list of frames
frames = sorted(glob.glob('frames_lr/*.png'))
total_frames = len(frames)

print(f"\n🔍 PASS 1: Upscaling {total_frames} frames (4x)...")
print(f"   {orig_w}x{orig_h} → {orig_w*4}x{orig_h*4}\n")

# Process frames - First 4x pass
for frame_path in tqdm(frames, desc="4x Upscale", unit="frame"):
    img = cv2.imread(frame_path, cv2.IMREAD_UNCHANGED)
    output, _ = upsampler_x4.enhance(img, outscale=4)
    output_path = frame_path.replace('frames_lr', 'frames_4x')
    cv2.imwrite(output_path, output)

# Free memory
del upsampler_x4, model_x4
gc.collect()
torch.cuda.empty_cache()

print("\n✅ First pass (4x) complete!")

## Step 6: Second Pass - 2x Upscale (for 8x total)

In [ ]:
if UPSCALE_FACTOR == 8:
    # Initialize Real-ESRGAN x2 for second pass
    print("🔧 Initializing Real-ESRGAN x2 for second pass...")
    
    model_x2 = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
    
    upsampler_x2 = RealESRGANer(
        scale=2,
        model_path='models/RealESRGAN_x2plus.pth',
        model=model_x2,
        tile=TILE_SIZE,
        tile_pad=10,
        pre_pad=0,
        half=True,
        device='cuda'
    )
    
    print(f"✅ Real-ESRGAN x2 ready!")
    
    # Get 4x upscaled frames
    frames_4x = sorted(glob.glob('frames_4x/*.png'))
    
    print(f"\n🔍 PASS 2: Upscaling {len(frames_4x)} frames (2x)...")
    print(f"   {orig_w*4}x{orig_h*4} → {orig_w*8}x{orig_h*8}\n")
    
    # Process frames - Second 2x pass
    for frame_path in tqdm(frames_4x, desc="2x Upscale", unit="frame"):
        img = cv2.imread(frame_path, cv2.IMREAD_UNCHANGED)
        output, _ = upsampler_x2.enhance(img, outscale=2)
        output_path = frame_path.replace('frames_4x', 'frames_8x')
        cv2.imwrite(output_path, output)
    
    # Free memory
    del upsampler_x2, model_x2
    gc.collect()
    torch.cuda.empty_cache()
    
    print("\n✅ Second pass (2x) complete! Total: 8x upscale achieved!")
    HR_FRAMES_DIR = 'frames_8x'
else:
    print("ℹ️ Skipping second pass (4x mode)")
    HR_FRAMES_DIR = 'frames_4x'

# Show result
sample_hr = cv2.imread(sorted(glob.glob(f'{HR_FRAMES_DIR}/*.png'))[0])
hr_h, hr_w = sample_hr.shape[:2]
print(f"\n📊 Upscale Complete:")
print(f"   Original: {orig_w}x{orig_h}")
print(f"   Upscaled: {hr_w}x{hr_h} ({UPSCALE_FACTOR}x)")

## Step 7: Frame Interpolation to 60 FPS with RIFE

In [ ]:
import sys
import math

# Calculate interpolation passes needed
fps_multiplier = TARGET_FPS / ORIGINAL_FPS
rife_passes = max(1, int(math.ceil(math.log2(fps_multiplier))))
actual_target_fps = ORIGINAL_FPS * (2 ** rife_passes)

print(f"📊 Frame Interpolation Plan:")
print(f"   Original FPS: {ORIGINAL_FPS:.2f}")
print(f"   Target FPS: {TARGET_FPS}")
print(f"   RIFE passes: {rife_passes} ({2**rife_passes}x frame multiplier)")
print(f"   Intermediate FPS: {actual_target_fps:.2f}")

if actual_target_fps > TARGET_FPS:
    print(f"   (Will adjust to exact {TARGET_FPS} FPS in final encoding)")

In [ ]:
# Create a temporary video from HR frames for RIFE processing
print(f"\n🎞️ Preparing frames for RIFE interpolation...")

# RIFE works better with video input, so we'll create a temp video
!ffmpeg -y \
    -framerate {ORIGINAL_FPS} \
    -i "{HR_FRAMES_DIR}/frame_%07d.png" \
    -c:v libx264 \
    -preset ultrafast \
    -crf 10 \
    -pix_fmt yuv420p \
    temp_hr_video.mp4 \
    -hide_banner -loglevel error

print("✅ Temporary HR video created")

In [ ]:
# Run RIFE interpolation
print(f"\n🎞️ Interpolating to {TARGET_FPS} FPS with RIFE AI...")
print("   Creating smooth motion between frames using deep learning.\n")

%cd /content/Practical-RIFE

# Run RIFE with specified multiplier
!python inference_video.py \
    --video /content/temp_hr_video.mp4 \
    --output /content/temp_interpolated.mp4 \
    --exp {rife_passes}

%cd /content

print("\n✅ Frame interpolation complete!")

## Step 8: Create Final Video

In [ ]:
print("🎬 Creating final video...")

# Check if RIFE output exists
if os.path.exists('temp_interpolated.mp4'):
    source_video = 'temp_interpolated.mp4'
    print(f"   Using RIFE interpolated video")
else:
    source_video = 'temp_hr_video.mp4'
    print("⚠️ Using upscaled video without interpolation")

# Re-encode to target FPS with high quality
print(f"   Encoding to {TARGET_FPS} FPS with CRF {CRF_QUALITY}...")

!ffmpeg -y \
    -i "{source_video}" \
    -vf "fps={TARGET_FPS}" \
    -c:v libx264 \
    -preset slow \
    -crf {CRF_QUALITY} \
    -pix_fmt yuv420p \
    -an \
    temp_final_no_audio.mp4 \
    -hide_banner -loglevel error

print("✅ Video encoded!")

In [ ]:
# Add audio from original video
print("🔊 Adding audio from original video...")

# Check if original has audio
result = subprocess.run(
    ['ffprobe', '-v', 'error', '-select_streams', 'a',
     '-show_entries', 'stream=codec_type', '-of', 'csv=p=0', INPUT_VIDEO],
    capture_output=True, text=True
)

if 'audio' in result.stdout:
    !ffmpeg -y \
        -i temp_final_no_audio.mp4 \
        -i "{INPUT_VIDEO}" \
        -c:v copy \
        -c:a aac -b:a 192k \
        -map 0:v:0 \
        -map 1:a:0 \
        -shortest \
        "{OUTPUT_VIDEO}" \
        -hide_banner -loglevel error
    print("✅ Audio added!")
else:
    !mv temp_final_no_audio.mp4 "{OUTPUT_VIDEO}"
    print("ℹ️ No audio in original video")

## Step 9: Download Your Ultra HD Video! 🎉

In [ ]:
print("="*70)
print("🎉 SUCCESS! Ultra HD Video Processing Complete!")
print("="*70)

# Show comparison
print("\n📊 Before vs After:")
print(f"\n   ORIGINAL:")
!ffprobe -v error -select_streams v:0 \
    -show_entries stream=width,height,r_frame_rate \
    -of default=noprint_wrappers=1 "{INPUT_VIDEO}"

print(f"\n   UPSCALED ({UPSCALE_FACTOR}x + {TARGET_FPS} FPS):")
!ffprobe -v error -select_streams v:0 \
    -show_entries stream=width,height,r_frame_rate \
    -of default=noprint_wrappers=1 "{OUTPUT_VIDEO}"

# File sizes
import os
orig_size = os.path.getsize(INPUT_VIDEO) / (1024 * 1024)
out_size = os.path.getsize(OUTPUT_VIDEO) / (1024 * 1024)
print(f"\n📁 File Size:")
print(f"   Original: {orig_size:.2f} MB")
print(f"   Upscaled: {out_size:.2f} MB")

# Download
print("\n" + "="*70)
print("📥 Starting download...")
from google.colab import files
files.download(OUTPUT_VIDEO)

## 🧹 Cleanup (Optional)

Run this to free up disk space:

In [ ]:
# Cleanup temporary files
!rm -rf frames_lr frames_4x frames_8x frames_interpolated
!rm -f temp_hr_video.mp4 temp_interpolated.mp4 temp_final_no_audio.mp4
print("✅ Temporary files cleaned up!")

# Show remaining disk space
!df -h /content

---

## 📝 Technical Notes

### 8x Upscaling Process
- **Pass 1**: Real-ESRGAN x4plus (trained on real-world degradations)
- **Pass 2**: Real-ESRGAN x2plus (fine detail enhancement)
- **Total**: 4x × 2x = **8x** resolution increase

### 60 FPS Interpolation
- **RIFE** (Real-Time Intermediate Flow Estimation v4.6)
- Uses optical flow AI to create intermediate frames
- Produces smooth, natural-looking motion

### Example Transformations
| Input | 8x Upscale | + 60 FPS |
|-------|------------|----------|
| 480×270 @ 24fps | 3840×2160 (4K) | 60 FPS |
| 640×360 @ 30fps | 5120×2880 (5K) | 60 FPS |
| 854×480 @ 25fps | 6832×3840 (7K) | 60 FPS |

### Tips for Best Results
1. **Source Quality**: Higher quality input = better output
2. **GPU Memory**: Reduce TILE_SIZE if you get OOM errors
3. **CRF Quality**: 15-18 for high quality, 23-28 for smaller files
4. **Processing Time**: ~2-10 min per minute of video (depends on resolution)

---

Made by Md. Mahir Labib